<a href="https://colab.research.google.com/github/pulkit8690/Sampling/blob/main/Sampling_102103267.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [ ]:
from imblearn.over_sampling import RandomOverSampler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.preprocessing import MinMaxScaler,StandardScaler
from sklearn.metrics import accuracy_score

In [ ]:
url = "https://raw.githubusercontent.com/AnjulaMehto/Sampling_Assignment/main/Creditcard_data.csv"
df = pd.read_csv(url)

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
sns.countplot(data= df, x= 'Class')

In [ ]:
df['Class'].value_counts()

In [ ]:
X= df.drop('Class',axis=1)
y= df['Class']

ros= RandomOverSampler(random_state=42)

X_resampled, y_resampled= ros.fit_resample(X,y)

resampled_df= pd.concat([pd.DataFrame(X_resampled,columns= X.columns), pd.Series(y_resampled,name= 'Class')], axis=1)

sns.countplot(data= resampled_df, x= 'Class')

In [ ]:
resampled_df['Class'].value_counts()

In [ ]:
z= 1.96
p= 0.5
E= 0.05

In [ ]:
sample_size= int(np.round((z**2 * p * (1-p)) / (E**2)))
sample_size

In [ ]:
random_sample= resampled_df.sample(n= sample_size, replace= False).reset_index(drop= True)

In [ ]:
random_sample.head()

In [ ]:
k= 5
sample_size= len(resampled_df)//k
sample_size

In [ ]:
indices= np.arange(0,len(resampled_df),k)[:sample_size]
systematic_sample= resampled_df.loc[indices].reset_index(drop= True)

In [ ]:
z= 1.96
p= 0.7
E= 0.01
S= 2

In [ ]:
sample_size= int(np.round((z**2 * p * (1-p)) / (E/S)**2))
sample_size

In [ ]:
len(resampled_df)

In [ ]:
stratified_sample= resampled_df.groupby(by= 'Class', group_keys= False).apply(lambda x: x.sample(n= sample_size//2, replace= True)).reset_index(drop= True)

In [ ]:
stratified_sample.head()

In [ ]:
train_stratified_sample, test_stratified_sample= train_test_split(resampled_df, test_size= 0.3, stratify= resampled_df['Class'])

In [ ]:
train_stratified_sample= train_stratified_sample.reset_index(drop= True)
test_stratified_sample= test_stratified_sample.reset_index(drop= True)

In [ ]:
train_stratified_sample

In [ ]:
test_stratified_sample

In [ ]:
X_test= test_stratified_sample.drop('Class',axis=1)
y_test= test_stratified_sample['Class']

In [ ]:
z= 1.96
p= 0.5
E= 0.05
C= 50

In [ ]:
sample_size= int(np.round((z**2 * p * (1-p)) / (E/C)**2))
sample_size

In [ ]:
clusters= [0,1]
sample_size= 500
cluster_sample= pd.DataFrame()

for cluster in clusters:
    cluster_df= resampled_df[resampled_df['Class']==cluster]
    sample= cluster_df.sample(n= sample_size)
    cluster_sample= pd.concat([cluster_sample,sample])

In [ ]:
cluster_sample= cluster_sample.reset_index(drop= True)

In [ ]:
cluster_sample

In [ ]:
num_bootstrap_samples = 1

for _ in range(num_bootstrap_samples):
    bootstrap_sample = resampled_df.sample(n=len(resampled_df), replace=True)

In [ ]:
bootstrap_sample= bootstrap_sample.reset_index(drop= True)

In [ ]:
bootstrap_sample

In [ ]:
lr_model= LogisticRegression(max_iter= 10000, solver= 'sag')
knn_model= KNeighborsClassifier(n_neighbors= 3)
rf_model= RandomForestClassifier()
dt_model= DecisionTreeClassifier()
nb_model= GaussianNB()

In [ ]:
models= [lr_model,knn_model,rf_model,dt_model,nb_model]
model_names= ['Logistic Regression','KNN','Random Forest','Decision Tree','Naive Bayes']

In [ ]:
samples= [random_sample, systematic_sample, train_stratified_sample, cluster_sample, bootstrap_sample]
sample_names= ['Simple Random Sample', 'Systematic Sample', 'Stratified Sample', 'Cluster Sample', 'Bootstrap Sample']

In [ ]:
test_samples= []

In [ ]:
for sample in samples:
    scaler= StandardScaler()
    X_train= scaler.fit_transform(sample.drop('Class',axis=1))
    X_train= pd.DataFrame(X_train, columns= df.columns[:-1])
    sample= pd.concat([X_train,sample['Class']],axis=1)
    X_test= scaler.fit_transform(X_test)
    X_test= pd.DataFrame(X_test, columns= df.columns[:-1])
    test_samples.append(X_test)

In [ ]:
accuracy_table= pd.DataFrame(index= model_names, columns= sample_names)

for i,model in enumerate(models):
    for j,sample in enumerate(samples):
        model.fit(sample.drop('Class',axis=1), sample['Class'])
        pred= model.predict(test_samples[j])
        accuracy_table.loc[model_names[i], sample_names[j]]= accuracy_score(y_test, pred)

In [ ]:
accuracy_table

In [ ]:
accuracy_table.to_csv('/content/accuracy_table.csv', index=True)